In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import os
import datetime as dt
import xgboost as xgb
from sklearn.svm import OneClassSVM
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import catboost
import sklearn
from sklearn.model_selection import cross_val_score, KFold, learning_curve, cross_validate, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import operator
import matplotlib.pyplot as plt
import xgboost as xgb
import os
import seaborn as sns
import datetime
from time import strftime, gmtime, time
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

tqdm.pandas()

In [3]:
sns.set_context("poster") 
plt.style.use('seaborn-poster')

In [4]:
# !pip install xgboost
# !pip install sklearn

In [5]:
def feature_importances(model) -> pd.DataFrame:

    importance_dict = {}

    for feature, importance in zip(X_train.columns, model.feature_importances_):
        importance_dict[feature] = importance

    df_importance = pd.DataFrame(sorted(importance_dict.items(), key=operator.itemgetter(1)))
    df_importance = df_importance.set_index(0).sort_values(by=[1], ascending=False)

    return df_importance

In [117]:
cat_tester = pd.read_csv("Parking_added_tester.csv")
cat_tester_1 = pd.read_csv("All_data_df.csv")
cat_tester_1

cat_tester_1 = cat_tester_1[cat_tester_1['Datetime'] > '2021']
cat_tester_1
# park specifier
# cat_tester_1 = cat_tester_1[cat_tester_1['Location']== 'Oosterpark']


,Unnamed: 0,Datetime,Location,Visits,Date,Time,Journeys,Windspeed,Temperature,Clouds,...,Day_x,Holiday_name,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,stringency_index
87323,87323,2021-01-01 00:00:00,Amstelpark,25.0,2021-01-01,00:00:00,21.818182,2.0,-0.4,1.00,...,1,227.412888,2021-01-01,-84.0,-64.0,-41.0,-80.0,-81.0,23.0,78.70
87324,87324,2021-01-01 00:00:00,Beatrixpark,154.0,2021-01-01,00:00:00,88.000000,2.0,-0.4,1.00,...,1,227.412888,2021-01-01,-84.0,-64.0,-41.0,-80.0,-81.0,23.0,78.70
87325,87325,2021-01-01 00:00:00,Erasmuspark,323.0,2021-01-01,00:00:00,13.933333,2.0,-0.4,1.00,...,1,227.412888,2021-01-01,-84.0,-64.0,-41.0,-80.0,-81.0,23.0,78.70
87326,87326,2021-01-01 00:00:00,Flevopark,65.0,2021-01-01,00:00:00,61.777778,2.0,-0.4,1.00,...,1,227.412888,2021-01-01,-84.0,-64.0,-41.0,-80.0,-81.0,23.0,78.70
87327,87327,2021-01-01 00:00:00,Gaasperpark,25.0,2021-01-01,00:00:00,15.400000,2.0,-0.4,1.00,...,1,227.412888,2021-01-01,-84.0,-64.0,-41.0,-80.0,-81.0,23.0,78.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724612,724612,2022-01-06 14:45:00,Westerpark West,91.0,2022-01-06,14:45:00,10.000000,5.0,5.4,3.25,...,6,227.412888,2022-01-06,-62.0,-12.0,-25.0,-59.0,-55.0,18.0,69.44
724613,724613,2022-01-06 15:00:00,Flevopark,165.0,2022-01-06,15:00:00,18.870370,5.0,5.2,4.00,...,6,227.412888,2022-01-06,-62.0,-12.0,-25.0,-59.0,-55.0,18.0,69.44
724614,724614,2022-01-06 15:00:00,Sarphatipark,776.0,2022-01-06,15:00:00,115.074419,5.0,5.2,4.00,...,6,227.412888,2022-01-06,-62.0,-12.0,-25.0,-59.0,-55.0,18.0,69.44
724615,724615,2022-01-06 15:00:00,Sloterpark,366.0,2022-01-06,15:00:00,23.804651,5.0,5.2,4.00,...,6,227.412888,2022-01-06,-62.0,-12.0,-25.0,-59.0,-55.0,18.0,69.44


In [79]:
# X, y = cat_tester.loc[:, cat_tester.columns != "Visits"], cat_tester[["Visits"]]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# cat_features = ['Date_x','Time','Location']

# X_train = X_train[['Location', 'Date_x', 'Time', 'Journeys' ,'Windspeed', 'Temperature',
#        'Clouds', 'Rain amount', 'Rain duration', 'Sun duration', 'Fog', 'Rain',
#        'Snow', 'Thunder', 'Ice', 'Holiday_Count', 'Year', 'Month', 'Day',
#        'Holiday_name', 'Holiday_Name_Ascension Thursday',
#        'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
#        'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
#        'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
#        'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
#        'Holiday_Name_Whit', 'Holiday_Name_Winter holiday','capacity.filled']]
# X_test = X_test[['Location', 'Date_x', 'Time', 'Journeys', 'Windspeed', 'Temperature',
#        'Clouds', 'Rain amount', 'Rain duration', 'Sun duration', 'Fog', 'Rain',
#        'Snow', 'Thunder', 'Ice', 'Holiday_Count', 'Year', 'Month', 'Day',
#        'Holiday_name', 'Holiday_Name_Ascension Thursday',
#        'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
#        'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
#        'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
#        'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
#        'Holiday_Name_Whit', 'Holiday_Name_Winter holiday','capacity.filled']]

# assert "ARRIVAL_DELAY" not in X_train.columns
# assert y_train.shape[1] == 1

In [80]:
# X, y = cat_tester_1.loc[:, cat_tester_1.columns != "Visits"], cat_tester_1[["Visits"]]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# cat_features = ['Date','Time','Location']

# X_train = X_train[['Location', 'Date', 'Time', 'Journeys' ,'Windspeed', 'Temperature',
#        'Clouds', 'Rain amount', 'Rain duration', 'Sun duration', 'Fog', 'Rain',
#        'Snow', 'Thunder', 'Ice', 'Holiday_Count', 'Year', 'Month', 'Day',
#        'Holiday_name', 'Holiday_Name_Ascension Thursday',
#        'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
#        'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
#        'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
#        'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
#        'Holiday_Name_Whit', 'Holiday_Name_Winter holiday']]
# X_test = X_test[['Location', 'Date', 'Time', 'Journeys', 'Windspeed', 'Temperature',
#        'Clouds', 'Rain amount', 'Rain duration', 'Sun duration', 'Fog', 'Rain',
#        'Snow', 'Thunder', 'Ice', 'Holiday_Count', 'Year', 'Month', 'Day',
#        'Holiday_name', 'Holiday_Name_Ascension Thursday',
#        'Holiday_Name_Easter', 'Holiday_Name_Fall holiday',
#        'Holiday_Name_Good Friday', 'Holiday_Name_Kings day',
#        'Holiday_Name_Liberation Day', 'Holiday_Name_May holiday',
#        'Holiday_Name_Spring holiday', 'Holiday_Name_Summer holiday',
#        'Holiday_Name_Whit', 'Holiday_Name_Winter holiday']]

# assert "ARRIVAL_DELAY" not in X_train.columns
# assert y_train.shape[1] == 1

In [81]:
X, y = cat_tester_1.loc[:, cat_tester_1.columns != "Visits"], cat_tester_1[["Visits"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
cat_features = ['Datetime','Date','Time','Location']

X_train = X_train[['Datetime', 'Location', 'Date', 'Time',
       'Journeys', 'Windspeed', 'Temperature', 'Clouds', 'Rain amount',
       'Rain duration', 'Sun duration', 'Fog', 'Rain', 'Snow', 'Thunder',
       'Ice', 'Holiday_Count', 'Year', 'Month', 'Day_x', 'Holiday_name',
        'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'stringency_index']]
X_test = X_test[['Datetime', 'Location', 'Date', 'Time',
       'Journeys', 'Windspeed', 'Temperature', 'Clouds', 'Rain amount',
       'Rain duration', 'Sun duration', 'Fog', 'Rain', 'Snow', 'Thunder',
       'Ice', 'Holiday_Count', 'Year', 'Month', 'Day_x', 'Holiday_name',
       'retail_and_recreation_percent_change_from_baseline',
       'grocery_and_pharmacy_percent_change_from_baseline',
       'parks_percent_change_from_baseline',
       'transit_stations_percent_change_from_baseline',
       'workplaces_percent_change_from_baseline',
       'residential_percent_change_from_baseline', 'stringency_index']]

assert "ARRIVAL_DELAY" not in X_train.columns
assert y_train.shape[1] == 1

In [134]:
locations = cat_tester_1.Location.unique()
# for x in locations:

for x in locations:
    test = cat_tester_1[cat_tester_1['Location']== x]
    
    X, y = test.loc[:, test.columns != "Visits"], test[["Visits"]]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    cat_features = ['Datetime','Date','Time','Location']

    X_train = X_train[['Datetime', 'Location', 'Date', 'Time',
           'Journeys', 'Windspeed', 'Temperature', 'Clouds', 'Rain amount',
           'Rain duration', 'Sun duration', 'Fog', 'Rain', 'Snow', 'Thunder',
           'Ice', 'Holiday_Count', 'Year', 'Month', 'Day_x', 'Holiday_name',
            'retail_and_recreation_percent_change_from_baseline',
           'grocery_and_pharmacy_percent_change_from_baseline',
           'parks_percent_change_from_baseline',
           'transit_stations_percent_change_from_baseline',
           'workplaces_percent_change_from_baseline',
           'residential_percent_change_from_baseline', 'stringency_index']]
    X_test = X_test[['Datetime', 'Location', 'Date', 'Time',
           'Journeys', 'Windspeed', 'Temperature', 'Clouds', 'Rain amount',
           'Rain duration', 'Sun duration', 'Fog', 'Rain', 'Snow', 'Thunder',
           'Ice', 'Holiday_Count', 'Year', 'Month', 'Day_x', 'Holiday_name',
           'retail_and_recreation_percent_change_from_baseline',
           'grocery_and_pharmacy_percent_change_from_baseline',
           'parks_percent_change_from_baseline',
           'transit_stations_percent_change_from_baseline',
           'workplaces_percent_change_from_baseline',
           'residential_percent_change_from_baseline', 'stringency_index']]

    assert "ARRIVAL_DELAY" not in X_train.columns
    assert y_train.shape[1] == 1
    
    cat_scores = {}
    for iterations in range(25, 550, 500):
    # for iterations in range(25, 500, 50):

#         print(f"Fitting model for {iterations} iterations...")

        start_time = time()

        cat_scores[iterations] = {}

        cat_model = catboost.CatBoostRegressor(iterations=iterations, cat_features=cat_features, verbose=False)

        cat_model.fit(X_train, y_train, eval_set=(X_test, y_test))


        cat_scores[iterations]["RMSE train"] = cat_model.get_best_score()["learn"]["RMSE"]  
        pred = cat_model.predict(X_train)
        mae_train = mean_absolute_error(y_train, pred) 
        cat_scores[iterations]['MAE Train'] = mae_train

        cat_scores[iterations]["RMSE test"] = cat_model.get_best_score()["validation"]["RMSE"]
        pred = cat_model.predict(X_test)
        mae_val = mean_absolute_error(y_test, pred)
        cat_scores[iterations]['MAE Test'] = mae_val

        cat_scores[iterations]["time"] = time() - start_time
        
    print(f"Scores for {x}")
#         print(f"Fitting model for {iterations} iterations took {time() - start_time} seconds...")
    print(cat_scores)

Scores for Amstelpark
{25: {'RMSE train': 43.11063970521807, 'MAE Train': 24.819551613421762, 'RMSE test': 45.04455074140204, 'MAE Test': 25.167580745423347, 'time': 0.17838406562805176}, 525: {'RMSE train': 29.487942174209177, 'MAE Train': 18.13749746785004, 'RMSE test': 36.39395323568072, 'MAE Test': 20.250648487231313, 'time': 6.188109874725342}}
Scores for Beatrixpark
{25: {'RMSE train': 179.82981268882688, 'MAE Train': 115.54686147027536, 'RMSE test': 186.8178555573701, 'MAE Test': 120.7166855479359, 'time': 0.17441892623901367}, 525: {'RMSE train': 140.49779518672824, 'MAE Train': 88.6503508569315, 'RMSE test': 161.7998646838482, 'MAE Test': 100.48534242118147, 'time': 5.985957145690918}}
Scores for Erasmuspark
{25: {'RMSE train': 80.9078628466417, 'MAE Train': 45.287822881995204, 'RMSE test': 79.71318012699732, 'MAE Test': 46.676058646739456, 'time': 0.18043804168701172}, 525: {'RMSE train': 59.09398417688391, 'MAE Train': 34.83361996423471, 'RMSE test': 64.30370612001256, 'MAE 

In [199]:
locations = cat_tester_1.Location.unique()
# for x in locations:
park_name = []
Iterations = []
RMSE_Train_Scores = []
MAE_Train_scores = []
RMSE_Test_Scores = []
MAE_Test_Scores = []
Run_time = []

for x in locations:
    test = cat_tester_1[cat_tester_1['Location']== x]
    
    X, y = test.loc[:, test.columns != "Visits"], test[["Visits"]]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    cat_features = ['Datetime','Date','Time','Location']

    X_train = X_train[['Datetime', 'Location', 'Date', 'Time',
           'Journeys', 'Windspeed', 'Temperature', 'Clouds', 'Rain amount',
           'Rain duration', 'Sun duration', 'Fog', 'Rain', 'Snow', 'Thunder',
           'Ice', 'Holiday_Count', 'Year', 'Month', 'Day_x', 'Holiday_name',
            'retail_and_recreation_percent_change_from_baseline',
           'grocery_and_pharmacy_percent_change_from_baseline',
           'parks_percent_change_from_baseline',
           'transit_stations_percent_change_from_baseline',
           'workplaces_percent_change_from_baseline',
           'residential_percent_change_from_baseline', 'stringency_index']]
    X_test = X_test[['Datetime', 'Location', 'Date', 'Time',
           'Journeys', 'Windspeed', 'Temperature', 'Clouds', 'Rain amount',
           'Rain duration', 'Sun duration', 'Fog', 'Rain', 'Snow', 'Thunder',
           'Ice', 'Holiday_Count', 'Year', 'Month', 'Day_x', 'Holiday_name',
           'retail_and_recreation_percent_change_from_baseline',
           'grocery_and_pharmacy_percent_change_from_baseline',
           'parks_percent_change_from_baseline',
           'transit_stations_percent_change_from_baseline',
           'workplaces_percent_change_from_baseline',
           'residential_percent_change_from_baseline', 'stringency_index']]

    assert "ARRIVAL_DELAY" not in X_train.columns
    assert y_train.shape[1] == 1
    
    cat_scores = {}
    
    for iterations in range(25, 1050, 1000):
    # for iterations in range(25, 500, 50):

#         print(f"Fitting model for {iterations} iterations...")

        start_time = time()

        cat_scores[iterations] = {}

        cat_model = catboost.CatBoostRegressor(iterations=iterations, cat_features=cat_features, verbose=False)

        cat_model.fit(X_train, y_train, eval_set=(X_test, y_test))


        cat_scores[iterations]["RMSE train"] = cat_model.get_best_score()["learn"]["RMSE"]  
        pred = cat_model.predict(X_train)
        mae_train = mean_absolute_error(y_train, pred) 
        cat_scores[iterations]['MAE Train'] = mae_train
        
        cat_scores[iterations]["RMSE test"] = cat_model.get_best_score()["validation"]["RMSE"]
        pred = cat_model.predict(X_test)
        mae_val = mean_absolute_error(y_test, pred)
        cat_scores[iterations]['MAE Test'] = mae_val

        cat_scores[iterations]["time"] = time() - start_time
        
        park_name.append(x)
        Iterations.append(iterations)
        RMSE_Train_Scores.append(cat_model.get_best_score()['learn'])
        MAE_Train_scores.append(mae_train)
        RMSE_Test_Scores.append(cat_model.get_best_score()['validation'])
        MAE_Test_Scores.append(mae_val)
        Run_time.append(time() - start_time)    
        
    print(f"Scores for {x}")
#         print(f"Fitting model for {iterations} iterations took {time() - start_time} seconds...")
    print(cat_scores)

Scores for Amstelpark
{25: {'RMSE train': 43.11063970521807, 'MAE Train': 24.819551613421762, 'RMSE test': 45.04455074140204, 'MAE Test': 25.167580745423347, 'time': 0.21419787406921387}, 1025: {'RMSE train': 27.632629856081916, 'MAE Train': 17.163519893201084, 'RMSE test': 35.605142934035946, 'MAE Test': 19.60207236976086, 'time': 13.896659851074219}}
Scores for Beatrixpark
{25: {'RMSE train': 179.82981268882688, 'MAE Train': 115.54686147027536, 'RMSE test': 186.8178555573701, 'MAE Test': 120.7166855479359, 'time': 0.2387990951538086}, 1025: {'RMSE train': 133.567408299325, 'MAE Train': 84.55975157274656, 'RMSE test': 158.50069143716476, 'MAE Test': 97.0085882213409, 'time': 14.043533086776733}}
Scores for Erasmuspark
{25: {'RMSE train': 80.9078628466417, 'MAE Train': 45.287822881995204, 'RMSE test': 79.71318012699732, 'MAE Test': 46.676058646739456, 'time': 0.2064518928527832}, 1025: {'RMSE train': 55.96824995026769, 'MAE Train': 33.28614775534097, 'RMSE test': 62.91755236349964, 'MA

In [163]:
# cat_scores

{25: {'RMSE train': 44.86375533516888,
  'MAE Train': 29.963320007019583,
  'RMSE test': 45.12612341040208,
  'MAE Test': 31.32664463266278,
  'time': 0.15749311447143555},
 525: {'RMSE train': 33.54155547075991,
  'MAE Train': 23.009790562258345,
  'RMSE test': 38.00321798516766,
  'MAE Test': 26.079773305696673,
  'time': 4.822803974151611}}

In [164]:
# cat_model.get_best_score()

{'learn': {'RMSE': 33.54155547075991},
 'validation': {'RMSE': 38.00321798516766}}

In [68]:
feature_importances(cat_model)

,1
0,
Location,35.706944
Time,16.495178
Journeys,10.765928
Date,10.478650
Month,5.685188
stringency_index,3.388832
Datetime,2.929532
Temperature,2.501718
workplaces_percent_change_from_baseline,2.209681


In [200]:
henk =pd.DataFrame(list(zip(park_name,Iterations, RMSE_Train_Scores, MAE_Train_scores, RMSE_Test_Scores, MAE_Test_Scores, Run_time)), 
                   columns=['Park_name','Iterations','RMSE_Train','MAE_Train', 'RMSE_Test','MAE_Test', 'Run_Time'])
for x in range(len(henk)):
    henk['RMSE_Train'][x] = henk['RMSE_Train'][x]['RMSE']
    henk['RMSE_Test'][x] = henk['RMSE_Test'][x]['RMSE']
henk = henk[henk['Iterations']==1025]
henk

/var/folders/3y/wg1nvwfd2fgfgy_dxxxkcbsw0000gn/T/ipykernel_17842/2460751921.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  henk['RMSE_Train'][x] = henk['RMSE_Train'][x]['RMSE']
/var/folders/3y/wg1nvwfd2fgfgy_dxxxkcbsw0000gn/T/ipykernel_17842/2460751921.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  henk['RMSE_Test'][x] = henk['RMSE_Test'][x]['RMSE']


,Park_name,Iterations,RMSE_Train,MAE_Train,RMSE_Test,MAE_Test,Run_Time
1,Amstelpark,1025,27.63263,17.163520,35.605143,19.602072,13.896736
3,Beatrixpark,1025,133.567408,84.559752,158.500691,97.008588,14.043545
5,Erasmuspark,1025,55.96825,33.286148,62.917552,37.439344,14.563862
7,Flevopark,1025,27.639806,18.389416,31.138231,20.441587,13.748829
9,Gaasperpark,1025,25.160999,16.533689,28.588033,18.745963,14.352997
11,Nelson Mandelapark,1025,43.539592,27.664815,49.853233,31.081558,14.439412
13,Noorderpark,1025,117.106111,74.634319,135.565121,83.810652,14.091785
15,Oosterpark,1025,125.365852,79.902796,149.337054,89.278475,14.860856
17,Park Frankendael,1025,58.784442,39.087364,63.715007,42.328319,14.582762
19,Sarphatipark,1025,97.971712,61.765802,111.447012,68.959338,13.745454


In [204]:
henk['RMSE_Test'].mean()

72.5388695335475

In [206]:
henk['MAE_Test'].mean()

45.623617516218225